Using two DataSets:

 1. DSNY_Monthly_Tonnage_Data.csv

 2. CD_Population_Estimates.csv

We are able to delinate the data needed to calculate and portray a multitude of important factors within the data.

We are also able to distinguish across these datasets the implications of the amount of refuse per borough in New York City has, when combined with population in a period which runs from 1992 through 2018.




In [2]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from scipy.stats import linregress
from pylab import rcParams
import matplotlib.patches as mpatches

In [3]:
# Read in Refuse DataFrame
# calculate year and month column based on date column

df = pd.read_csv("Resources/DSNY_Monthly_Tonnage_Data.csv")
df_pop = pd.read_csv("CD_Population_Estimates.csv")
df.head()

FileNotFoundError: [Errno 2] File b'CD_Population_Estimates.csv' does not exist: b'CD_Population_Estimates.csv'

In [ ]:
# Setting to these columns gets rid of the columns we do not want to use
    # from the DSNY_Monthly_Tonnage_Data.csv dataset

df = pd.read_csv("Resources/DSNY_Monthly_Tonnage_Data.csv", usecols=['MONTH',
                                                                     'BOROUGH', 
                                                                     'COMMUNITYDISTRICT',
                                                                     'REFUSETONSCOLLECTED',
                                                                     'MGPTONSCOLLECTED',
                                                                     'BOROUGH_ID'])

df

In [ ]:
df_pop.head(2)

In [ ]:
df_pop.columns

In [4]:
df.head(2)

,MONTH,BOROUGH,COMMUNITYDISTRICT,REFUSETONSCOLLECTED,PAPERTONSCOLLECTED,MGPTONSCOLLECTED,RESORGANICSTONS,SCHOOLORGANICTONS,LEAVESORGANICTONS,XMASTREETONS,BOROUGH_ID
0,1993 / 11,Manhattan,1,625.2,119.4,34.4,NaN,NaN,NaN,NaN,1
1,1995 / 09,Manhattan,1,633.1,117.2,45.8,NaN,NaN,NaN,NaN,1


In [5]:
df.columns

Index(['MONTH', 'BOROUGH', 'COMMUNITYDISTRICT', 'REFUSETONSCOLLECTED',
       'PAPERTONSCOLLECTED', 'MGPTONSCOLLECTED', 'RESORGANICSTONS',
       'SCHOOLORGANICTONS', 'LEAVESORGANICTONS', 'XMASTREETONS', 'BOROUGH_ID'],
      dtype='object')

In [6]:
df["Year"] = df["MONTH"].str[:4]
df["Month"] = df["MONTH"].str[-2:]

# Delete Un-wanted Years

df = df[df["Year"] !="1990"]
df = df[df["Year"] !="1991"]
df = df[df["Year"] !="2019"]
df = df[df["Year"] !="2020"]

# The index() method searches an element in the list and returns its index.
# group by borough and compute sum in order to get population
    # by borough for each year
# groupby(by = Used to determine the groups for the groupby.
    #If by is a function, it’s called on each value of the object’s index

df = df.groupby(by=["BOROUGH", "Year"]).sum()

# reset_index = resets index to numerical value

df.reset_index(inplace=True)


                             

In [7]:
df_pop = pd.read_csv("CD_Population_Estimates.csv")

FileNotFoundError: [Errno 2] File b'CD_Population_Estimates.csv' does not exist: b'CD_Population_Estimates.csv'

In [31]:
# Input borough and year as strings ouput, population in that borough,
# that year

df_borough_pop = df_pop.groupby(by="Borough").sum()


In [32]:
# Function that looks up population given borough and year
    # applied function to borough and year in each row of the Refuse DataFrame
    # Result of the function for the population in the borough for that year, appears 
        # in the right

def pop_lookup(borough, year):
    return df_borough_pop.loc[borough, year] # get location from borough and year
                                             # locator method allows to return a 
                                                 # specific value or column by labels


In [33]:
# df["Population"] adds column of Population
# df.apply( = applies function to DataFrame and applies it row by row
# run function pop_lookup
# lambda = A lambda function can take any number of arguments, but can only have one expression
# x: = variable that is passed in, which represents the DataFrame
# lambda will take series as input, applied row by row, looks up values and applies pop_lookup to those 2 values
# axis=1 = specifies the rows and not the columns

df["Population"] = df.apply( 
    lambda x: pop_lookup(x["BOROUGH"], x["Year"]), axis=1)

#df.head()

In [34]:
# Create column using dot notation because their are no spaces in column headers


df["PerCapitaRefuse"] = df.REFUSETONSCOLLECTED / df.Population

# prints all rows
print(df.loc[:,["BOROUGH",
                "Year",
                "REFUSETONSCOLLECTED",
                "Population",
                "PerCapitaRefuse"]])

           BOROUGH  Year  REFUSETONSCOLLECTED  Population  PerCapitaRefuse
0            Bronx  1992             470892.7   1214085.0         0.387858
1            Bronx  1993             427840.6   1230073.0         0.347817
2            Bronx  1994             440798.9   1242225.0         0.354846
3            Bronx  1995             438483.8   1253781.0         0.349729
4            Bronx  1996             438951.8   1264701.0         0.347080
..             ...   ...                  ...         ...              ...
130  Staten Island  2014             179931.5    470413.0         0.382497
131  Staten Island  2015             175985.0    470858.0         0.373754
132  Staten Island  2016             176080.7    472545.0         0.372622
133  Staten Island  2017             178575.7    473842.0         0.376868
134  Staten Island  2018             181969.4    474503.0         0.383495

[135 rows x 5 columns]


In [35]:
df["Year"]=df["Year"].map(int)
borough_gb = df.groupby("BOROUGH")

In [1]:
bronx = borough_gb.get_group("Bronx")
queens = borough_gb.get_group("Queens")
manhattan = borough_gb.get_group("Manhattan")
statenisland = borough_gb.get_group("Staten Island")
brooklyn = borough_gb.get_group("Brooklyn")
rvals = {"Bronx":None,"Queens":None,"Manhattan":None,"Staten Island":None,"Brooklyn":None}

slope, intercept, r_value, p_value, std_err = linregress(statenisland["Year"], statenisland["PerCapitaRefuse"])
x_vals = range(1992, 2019, 1)
y_vals = [slope * x + intercept for x in x_vals]
plt.plot(x_vals, y_vals, color="purple")
rvals["Staten Island"] = r_value

slope, intercept, r_value, p_value, std_err = linregress(bronx["Year"], bronx["PerCapitaRefuse"])
x_vals = range(1992, 2019, 1)
y_vals = [slope * x + intercept for x in x_vals]
plt.plot(x_vals, y_vals, color="red")
rvals["Bronx"] = r_value

slope, intercept, r_value, p_value, std_err = linregress(manhattan["Year"], manhattan["PerCapitaRefuse"])
x_vals = range(1992, 2019, 1)
y_vals = [slope * x + intercept for x in x_vals]
plt.plot(x_vals, y_vals, color="orange")
rvals["Manhattan"] = r_value

slope, intercept, r_value, p_value, std_err = linregress(brooklyn["Year"], brooklyn["PerCapitaRefuse"])
x_vals = range(1992, 2019, 1)
y_vals = [slope * x + intercept for x in x_vals]
plt.plot(x_vals, y_vals, color="blue")
rvals["Brooklyn"] = r_value

slope, intercept, r_value, p_value, std_err = linregress(queens["Year"], queens["PerCapitaRefuse"])
x_vals = range(1992, 2019, 1)
y_vals = [slope * x + intercept for x in x_vals]
plt.plot(x_vals, y_vals, color="green")
rvals["Queens"] = r_value

plt.scatter(brooklyn["Year"] ,brooklyn["PerCapitaRefuse"], label="Brooklyn", color="blue")
plt.scatter(bronx["Year"] ,bronx["PerCapitaRefuse"], label="Bronx", color="red")
plt.scatter(queens["Year"] ,queens["PerCapitaRefuse"], label="Queens", color="green")
plt.scatter(manhattan["Year"] ,manhattan["PerCapitaRefuse"], label="Manhattan", color="orange")
plt.scatter(statenisland["Year"] ,statenisland["PerCapitaRefuse"], label="Staten Island", color="purple")

rcParams['figure.figsize'] = 10, 10

plt.xlabel("Year")
plt.ylabel("Per Capita Refuse")
plt.title("Each Borough's Per Capital Refuse by Year")
plt.yticks(np.arange(0.20, 0.50, 0.02))
plt.legend(loc="best")
plt.savefig("BoroughCapitaRegressions.png")
plt.show

NameError: name 'borough_gb' is not defined